The purpose of this notebooks is to explore advanced queries of the STREAM data.

In [7]:
import dask.dataframe as dd
from utils import S3hsclient

In [2]:
hs = S3hsclient.S3HydroShare()

Please Enter Your HydroShare Credentials


Username:  TonyCastronova
Password for TonyCastronova:  ········


In [5]:
hs_paths = {
    "Anthropogenic": "tonycastronova/248ec0f13d6c4580b2faa66425cb58c3/data/contents/dynamic_antropogenic.parquet",
    "gauges": "tonycastronova/248ec0f13d6c4580b2faa66425cb58c3/data/contents/gauges.parquet",
    "Grab Samples": "tonycastronova/248ec0f13d6c4580b2faa66425cb58c3/data/contents/grab_samples.parquet",
    "Land Use/Cover": "tonycastronova/248ec0f13d6c4580b2faa66425cb58c3/data/contents/lulc.parquet",
    "Streamflow": "tonycastronova/248ec0f13d6c4580b2faa66425cb58c3/data/contents/streamflow.parquet",
    "water_quality": "tonycastronova/248ec0f13d6c4580b2faa66425cb58c3/data/contents/water_quality.parquet",
    "Historical Meteorology": "tonycastronova/248ec0f13d6c4580b2faa66425cb58c3/data/contents/dynamic_historical_meteorology",
    'HUC6': 'tonycastronova/248ec0f13d6c4580b2faa66425cb58c3/data/contents/huc6_dissolved.parquet',
    'Metadata': 'tonycastronova/248ec0f13d6c4580b2faa66425cb58c3/data/contents/metadata.parquet',
}

In [4]:
s3 = hs.get_s3_filesystem()

In [77]:
%%time 

# load our data
df_meta = dd.read_parquet(
    hs_paths['Metadata'],
    filesystem=s3
)

df_wq = dd.read_parquet(
    hs_paths['water_quality'],
    filesystem=s3
)


# filter by record size
min_sample_size = 20000
#hucs = ['020401', '020402']
result = df_meta[(df_meta["pH"] > min_sample_size) & \
                 (df_meta["WTemp_C"] > min_sample_size) & \
                 (df_meta["huc06"].isin(hucs))
                     ][['STREAM_ID']]

                  on="gauge",
                  how="inner")

dat = dat.compute()

first_gauge = dat["gauge"].iloc[0]
dat[dat["gauge"] == first_gauge]["pH"].plot();


CPU times: user 4.24 s, sys: 1.61 s, total: 5.85 s
Wall time: 14.7 s


In [78]:
dat

,DateTime,WTemp_C,Flag_WTemp_C,SpC_uScm,Flag_SpC_uScm,DO_mgL,Flag_DO_mgL,pH,Flag_pH,Turb_FNU,...,NO3_mgNL,Flag_NO3_mgNL,fDOM_QSU,Flag_fDOM_QSU,fDOM_RFU,Flag_fDOM_RFU,Chla_ugL,Flag_Chla_ugL,PC_RFU,Flag_PC_RFU
0,2007-10-01 05:00:00,17.300,A,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,...,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
1,2007-10-01 05:00:00,15.725,A,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,...,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
2,2007-10-01 05:00:00,13.925,A,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,...,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
3,2007-10-01 05:00:00,17.900,A,355.0,A,NaN,<NA>,7.6,A,2.00,...,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
4,2007-10-01 05:00:00,23.500,A,875.0,A,6.9,A,7.1,A,NaN,...,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9465816,2025-08-10 20:00:00,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,4.60,...,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
9465817,2025-08-10 20:00:00,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,3.10,...,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
9465818,2025-08-10 20:00:00,NaN,<NA>,NaN,<NA>,10.4,P,8.6,P,3.40,...,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
9465819,2025-08-10 20:00:00,NaN,<NA>,NaN,<NA>,NaN,<NA>,8.1,P,NaN,...,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>


In [71]:
import sys
sys.getsizeof(dat) / 1024**2

298.6960344314575

Notes

1. metadata counts could have a different naming convention so that it's not confusing when joining
2. gauge column still exists in parquets.